In [3]:
import pandas as pd
import numpy as np
import os
import random
import sys
import glob 
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from IPython.display import Audio
import sys
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import classification_report
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.regularizers import l2

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.multiclass import unique_labels
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb
import optuna

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from tqdm import tqdm
import time

In [7]:

# FUNCTION TO STRETCH THE SOUND
def stretch(x, rate=0.8):
    data = librosa.effects.time_stretch(x, rate)
    return data

# FUNCTION TO STRETCH THE SOUND
def stretch(x, rate=0.8):
    data = librosa.effects.time_stretch(x, rate)
    return data

# FUNCTION TO ADD WHITE NOISE
def noise(x):
    noise_amp = 0.05*np.random.uniform()*np.amax(x)   
    x = x.astype('float64') + noise_amp * np.random.normal(size=x.shape[0])
    return x


# FUNCTION FOR pre emphasis filter

import matplotlib.pyplot as plt
y, sr = librosa.load(librosa.ex('trumpet'))


# feature extract function

def extract_feature(file_name):
    """Function Extracts Features from WAV file"""
    orig, sample_rate = librosa.load(file_name)
    X = librosa.effects.preemphasis(orig)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) #Mel-frequency cepstral coefficients (MFCCs)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

# dict emotion labels & gender
#anger, disgust, fear, happiness, sadness, and surprise


emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

def gender(g):
    """Returns Gender Label"""
    if int(g[0:2]) % 2 == 0:
        return 'female'
    else:
        return 'male'


In [8]:
# extract name & path from files

import os
from fnmatch import fnmatch

root = "/Users/davidpitoun/All_Ironhack/Final Project/data/song_speech/"
pattern = "*.wav"

filenames = []
pathx = []
path_name = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            filenames.append(name)
            pathx.append(path)
            path_name.append(os.path.join(path, name))

In [10]:
for i in path_name:
    name = os.path.basename(i)
    yoo = name.split("-")[2] 
    if "01" in yoo:
        path_name.remove(i)
    if "02" in yoo:
        path_name.remove(i)

In [11]:
len(path_name)

2077

In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

def load_data(test_size=0.2):
    """Loads Data from directory containing WAV files."""
    x=[]
    y=[]
    for file in tqdm(path_name):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]] + '_' + gender(file_name.split("-")[-1])
        feature=extract_feature(file)
        x.append(feature)
        y.append(emotion)
        le.fit(y)# didnt apply
        y2 = le.transform(y)# didnt apply forgot to input y as y2
    return train_test_split(np.array(x), y2, test_size=test_size, random_state=30)

In [13]:
X_train, X_test, y_train, y_test = load_data()

100%|███████████████████████████████████████| 2077/2077 [08:45<00:00,  3.95it/s]


In [14]:
print((X_train.shape, X_test.shape))

((1661, 180), (416, 180))


In [15]:
print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 180


In [32]:
X_train

array([[-7.68133057e+02, -6.02271366e+00, -1.21641998e+01, ...,
         2.71150111e-06,  2.89187057e-07,  2.80117707e-09],
       [-6.18547668e+02,  9.51341152e+00, -4.64498825e+01, ...,
         1.40903485e-05,  1.16811498e-06,  8.60882476e-09],
       [-6.43316223e+02,  1.53809059e+00, -1.78000431e+01, ...,
         6.95666458e-05,  6.56706425e-06,  5.43515846e-08],
       ...,
       [-4.57340302e+02, -1.87711277e+01, -3.37659225e+01, ...,
         4.97421902e-03,  4.93399799e-04,  5.29437921e-06],
       [-5.68488647e+02, -3.45060768e+01, -3.07455120e+01, ...,
         8.97173362e-04,  7.25102509e-05,  1.47632329e-06],
       [-7.06416077e+02, -3.05436859e+01, -3.18417473e+01, ...,
         4.03238701e-05,  5.16455839e-06,  1.47998549e-07]])

In [ ]:
!pip install tpot

In [35]:
# define TPOTClassifier
tpotmodel = TPOTClassifier(
   generations=10,
   population_size=30,
   config_dict='TPOT light',
   cv=5,
   verbosity=2)

# performing the search for best fit

tpotmodel.fit(X_test, y_test)

# exporting best model
tpotmodel.export('tpot_spsdataunfilt.py')

Optimization Progress:   0%|          | 0/330 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4709982788296041

Generation 2 - Current best internal CV score: 0.48307515777395305

Generation 3 - Current best internal CV score: 0.48307515777395305

Generation 4 - Current best internal CV score: 0.48307515777395305

Generation 5 - Current best internal CV score: 0.4878944348823867

Generation 6 - Current best internal CV score: 0.4878944348823867

Generation 7 - Current best internal CV score: 0.5045897877223179

Generation 8 - Current best internal CV score: 0.5045897877223179

Generation 9 - Current best internal CV score: 0.5045897877223179

Generation 10 - Current best internal CV score: 0.5045897877223179

Best pipeline: LogisticRegression(MaxAbsScaler(input_matrix), C=5.0, dual=False, penalty=l2)


In [36]:
tpotmodel.predict(X_train)

array([13,  7,  9, ...,  9,  6, 12])

In [37]:
#  TPOTClassifier with Speech/Song & preemp,-neutral&calm
# logreg or mlp?
tpotmodel.score(X_test, y_test)


0.8341346153846154

In [157]:
print(tpotmodel.score(X_test, y_test))
tpotmodel.export('tpot_top.py')

0.8341346153846154


In [67]:
#  TPOTClassifier with Speech/Song & preemp,-neutral&calm
# 1sttpot with mlp
tpotmodel.score(X_test, y_test)


0.6354378818737271

In [62]:
clf = MLPClassifier(random_state=30, max_iter=300,alpha=0.1, learning_rate_init=0.001).fit(X_train1, y_train)

In [129]:
from sklearn.metrics import confusion_matrix

In [63]:
clf.predict(X_train1)
clf.score(X_test1,y_test)


0.7355769230769231

In [160]:
'''set back 2nd line to y_test'''
y_pred=clf.predict(X_test)
clf.score(X_test1,y_pred)


0.02403846153846154

In [132]:
confusion_matrix(y_test, y_pred)

array([[25,  2,  0,  0,  3,  0,  2,  0,  0,  0,  0,  0,  2,  0,  1,  0],
       [ 0, 33,  0,  0,  0,  3,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  6,  0,  0,  0,  1,  0,  0,  0,  0,  0,  3,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0,  0,  1,  0,  1,  0,  2,  0,  0],
       [ 0,  0,  0,  0, 15,  1,  1,  0,  1,  1,  0,  0,  2,  0,  1,  0],
       [ 0,  0,  0,  0,  1, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 6,  0,  0,  0,  2,  0, 14,  0,  3,  0,  0,  0, 12,  0,  3,  0],
       [ 0,  3,  0,  0,  0,  3,  0, 22,  0,  1,  0,  0,  0,  7,  0,  2],
       [ 0,  0,  0,  0,  0,  0,  1,  0, 29,  0,  0,  0,  0,  0,  1,  1],
       [ 1,  2,  0,  2,  0,  2,  0,  3,  0, 27,  0,  0,  0,  4,  0,  2],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  4,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  5,  0,  5,  0,  0],
       [ 1,  0,  0,  0,  2,  0,  9,  0,  1,  0,  1,  0, 32,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  2,  0,

In [ ]:
(array([15,  9,  0, 12, 12,  4,  9,  0,  2, 13,  7,  0, 14,  1, 13, 11,  8,
        4,  7, 12,  4,  0, 11,  7,  7,  3, 13,  8,  9,  4, 11, 12,  6,  6,
        7, 12,  1, 12, 12,  4,  1, 12,  1,  5,  6,  7,  1, 12,  0,  9,  8,
        6,  6,  1,  7,  2, 13,  9,  9, 15,  6,  7,  0,  0,  8,  1,  2, 11,
        6, 12, 12, 10,  5,  9,  7,  3,  6, 13, 15,  8,  9,  1,  0,  7, 11,
       14, 15,  4, 12, 10,  9, 15, 15, 15, 11, 14,  0, 10, 15,  6, 12,  5,
        6,  4,  8, 12, 13,  2,  1,  0,  6,  9,  8, 12,  0,  9,  6,  3, 13,
       15,  4,  6,  8,  1,  5,  6,  8, 12,  5, 15,  7,  1,  6,  1,  6, 12,
       14,  6,  8,  3,  8,  8,  0, 12,  1, 12,  9,  9, 13, 14, 12,  5,  1,
        8,  8,  6,  6,  6, 13, 12,  4, 14,  8, 11,  2, 13, 11,  9,  0, 12,
        4, 13,  0,  6,  9,  6,  7, 15, 14, 15,  6,  0,  6,  3,  9,  7,  8,
        7,  6,  1,  1,  7, 13, 11,  5, 12,  9,  0, 13,  1, 12, 13, 13,  1,
        9, 13,  0, 10, 12, 12, 14,  4, 12,  9,  0,  5,  7,  1,  8,  4, 12,
        0,  7, 13, 10,  8,  1,  8,  7,  0,  9,  0,  9, 13, 13,  0,  9, 13,
        7,  6,  9,  8,  4,  8,  2,  1, 13,  4, 12, 12,  1, 11, 12,  0,  7,
       10, 15, 12,  8,  9,  7,  9, 12,  1, 15,  6,  7,  8,  9, 13,  1, 12,
        4,  8,  0, 12,  9,  8,  7, 14,  8, 13, 13,  1,  9,  7, 15,  7,  3,
        9,  9,  5,  9, 12,  1,  2,  6,  7,  0, 12, 13,  1,  6,  6, 11,  5,
        8, 14,  8,  6,  1, 13, 10, 12,  7,  4,  0,  7,  9,  9,  4, 13,  9,
        8,  5,  0, 13,  7,  2,  9,  1,  8,  7,  6,  7,  7,  8,  9, 12,  7,
       13,  0,  7,  0,  5, 15,  6,  6, 12,  2, 15,  6,  9, 15,  9,  7,  8,
        6,  1,  6, 12,  4,  1, 13,  6,  7, 13, 12,  7, 14,  0, 12, 13, 12,
        0,  6, 13,  1,  0, 12,  1,  0,  4,  9,  0, 14, 15,  7, 12,  1,  4,
       14, 12,  6,  1, 13,  0, 15,  2,  5,  1,  9, 12, 13,  4, 15,  9, 13,
        9,  4,  1, 13, 11,  9,  1,  1]), array([15,  9,  0,  4,  6,  4,  9,  0,  2, 13, 13,  1, 14,  1, 13, 13,  8,
        4, 15, 12,  4,  0,  5,  7,  7,  3,  7,  8, 13,  4, 13, 12, 14,  8,
        7,  6,  9, 12, 12, 14,  1,  6,  1,  5,  6,  7,  1,  6,  0,  9,  8,
        6, 12,  1,  5,  2, 13,  9, 13, 15, 14, 15,  0,  0,  8,  1,  2, 11,
        6, 12,  6, 10,  5,  9,  7, 11, 12, 13, 13,  8,  9,  1,  0,  9, 13,
       14, 13,  4, 12, 12,  1, 15, 15,  9, 11, 14,  0, 10, 15, 12, 12,  5,
       14,  4,  8, 12, 13,  2,  5, 14,  0,  7,  8,  6,  0, 13, 12, 13, 13,
        9,  6,  6,  8,  1,  5,  4,  8,  8,  5,  9,  5,  1, 12,  1,  6, 12,
        8,  8, 14, 13,  8,  8,  0, 12,  1, 12,  5,  9, 13, 14, 12,  5,  1,
        8,  8, 12,  0,  6, 13, 12,  8, 14,  8,  7, 12, 13, 11,  9,  0, 12,
        5, 13,  6,  0,  9,  6,  7, 15, 12, 13,  6,  4,  6,  9, 13,  7,  8,
        5, 12,  1,  1, 13,  9, 13,  5,  6,  9,  0, 13,  1, 12,  7, 13,  5,
        9, 13,  0, 12, 12, 12, 10,  4, 12,  9,  0,  5,  7,  1,  8,  4, 12,
       12,  7, 11,  2,  8,  1,  8,  7,  1,  9,  0,  9, 13, 13,  0,  9, 13,
        7, 12,  9,  8,  4,  8, 12,  1, 13,  4, 12, 12,  1, 11, 12,  0,  7,
       10,  5, 14,  8,  7,  1,  9, 12,  1, 13,  6, 13,  6,  3, 13,  1, 12,
        4,  8,  4, 12,  9,  8, 13, 14,  8, 13,  7,  9,  5, 13, 15,  7,  5,
        9,  9,  5,  3, 12,  1,  2, 12,  7,  6, 12, 13,  1,  0,  8, 13,  4,
       15, 14,  8,  0,  1, 13, 10, 10,  7,  4,  0,  7,  9,  0,  4, 13,  9,
        8,  5,  0, 13, 13,  2,  9,  1,  8,  1, 12,  1,  7,  8,  9, 12, 13,
       13,  0,  7,  0, 15, 13,  6, 12, 12,  6, 15, 12, 15, 15,  9,  7,  8,
        6,  1,  6,  6, 12,  1, 13,  0,  7, 13,  0,  7, 14,  0, 12, 15, 14,
       12,  4, 15,  1,  4, 12,  1,  0, 12,  7,  0,  4,  7,  7,  6,  1,  9,
        8, 12,  6,  5, 13,  0,  5, 12,  5,  1,  9,  4, 13,  4, 13,  1,  9,
        9,  4,  1,  7, 11, 15,  9,  1]))

In [161]:
clf = MLPClassifier(random_state=30, max_iter=300,alpha=0.1, learning_rate_init=0.01).fit(X_train, y_train)
clf.predict(X_train1)
clf.score(X_test1,y_test)

0.3557692307692308

In [ ]:
0.6333592979176403	MLPClassifier(StandardScaler(input_matrix), MLPClassifier__alpha=0.1, MLPClassifier__learning_rate_init=0.001)

In [67]:
clf = MLPClassifier(random_state=30, max_iter=300,alpha=0.1, learning_rate_init=0.01).fit(X_train1, y_train)
clf.predict(X_train1)
clf.score(X_test1,y_test)

0.6682692307692307

In [57]:
# example of a standardization
'''
from sklearn.preprocessing import StandardScaler

# define standard scaler
scaler = StandardScaler()
# transform data
X_train1 = scaler.fit_transform(X_train)
X_test1= scaler.fit_transform(X_test)
'''


In [59]:
from sklearn.preprocessing import MaxAbsScaler
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    LogisticRegression(C=5.0, dual=False, penalty="l2")
)

exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_train1)
exported_pipeline.score(X_test1, y_test)

0.6370192307692307

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [64]:

# Average CV score on the training set was: 0.5492181021206018
exported_pipeline2 = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, max_features=0.5, min_samples_leaf=2, min_samples_split=11, n_estimators=100, subsample=0.5)
# Fix random state in exported estimator
if hasattr(exported_pipeline2, 'random_state'):
    setattr(exported_pipeline2, 'random_state', 20)

exported_pipeline2.fit(X_train1, y_train)
exported_pipeline2.predict(X_train1)


array([13,  9,  9, ...,  7,  6, 12])

In [65]:
exported_pipeline2.score(X_test1, y_test)

0.31971153846153844

In [ ]:
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        FunctionTransformer(copy)
    ),
    MinMaxScaler(),
    LogisticRegression(C=20.0, dual=False, penalty="l2")
)

exported_pipeline.fit(X_train, y_train)
exported_pipeline.predict(X_train)


In [73]:
mod = LogisticRegression(C=20.0, dual=False, penalty="l2")
mod.fit(X_train1, y_train)
mod.predict(X_train1)
mod.score(X_test1, y_test)

0.6057692307692307

In [44]:
from tpot import tpotClassifier


ModuleNotFoundError: No module named 'tpot.builtins'; 'tpot' is not a package

In [45]:
from sklearn.pipeline import make_union
from sklearn.preprocessing import FunctionTransformer
import copy
from sklearn.ensemble import StackingClassifier
from sklearn.base import BaseEstimator, TransformerMixin, is_classifier
from tpot.builtins import StackingEstimator
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tpot.builtins'; 'tpot' is not a package

In [ ]:
estimators = [DecisionTreeClassifier(criterion="entropy", max_depth=3, min_samples_leaf=11, min_samples_split=7)]

exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        StackingClassifier(estimators=estimators)
    ),
    PCA(iterated_power=9, svd_solver="randomized"),
    LogisticRegression(C=0.1, dual=False, penalty="l2")
)

exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_train1)
exported_pipeline.score(X_test1, y_test)

In [162]:
m=LogisticRegression().fit(X_train1, y_train)
m.predict(X_test1)
m.score(X_test1, y_test)

0.6177884615384616

In [135]:
estimators = [
    ('rf', DecisionTreeClassifier(criterion="entropy", max_depth=3, min_samples_leaf=11, min_samples_split=7)),
    ('svr', MLPClassifier(random_state=30, max_iter=300,alpha=0.1, learning_rate_init=0.001))]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression())


In [136]:
clf.fit(X_train1, y_train)

StackingClassifier(estimators=[('rf',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=3,
                                                       min_samples_leaf=11,
                                                       min_samples_split=7)),
                               ('svr',
                                MLPClassifier(alpha=0.1, max_iter=300,
                                              random_state=30))],
                   final_estimator=LogisticRegression())

In [ ]:
clf.train(X_train)

In [137]:
clf.score(X_test1, y_test)

0.7307692307692307

In [182]:

predictions = clf.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=clf.classes_)
disp.plot()


In [139]:
parameters = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.01,0.001,0.0015],
    'learning_rate': ['constant','adaptive'],
}

In [31]:
from sklearn.model_selection import GridSearchCV

In [141]:
mlp = MLPClassifier(random_state=30, max_iter=300)
est = GridSearchCV(mlp, parameters)
est.fit(X_train1, y_train)
est.predict(X_train1)
est.score(X_test1,y_test)

0.7067307692307693

In [ ]:
gridS_ResMlp = pd.DataFrame(est.cv_results_)
gridS_ResMlp.head()

In [148]:
est.best_estimator_

MLPClassifier(alpha=0.01, max_iter=300, random_state=30)

In [149]:
est.best_score_

0.7098086037845073

In [143]:
exported_pipelineB = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, max_features=0.5, min_samples_leaf=2, min_samples_split=11, n_estimators=100, subsample=0.5)
# Fix random state in exported estimator
if hasattr(exported_pipelineB, 'random_state'):
    setattr(exported_pipelineB, 'random_state', 20)

exported_pipelineB.fit(X_train1, y_train)
exported_pipelineB.predict(X_train1)
est.score(X_test1,y_test)

0.7067307692307693

In [ ]:
# best params for MLP classifier, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}

# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

In [30]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [153]:
# Gaussian Model


kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0)
gpc.fit(X_train1, y_train)
gpc.predict(X_test1)
gpc.score(X_test1, y_test)

0.6706730769230769

In [29]:
from sklearn.feature_selection import SelectFwe, chi2
from sklearn.feature_selection import SelectFpr, f_classif

In [167]:
exported_pipeline = make_pipeline(
    StandardScaler(),
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="gini", max_depth=1, min_samples_leaf=11, min_samples_split=19)),
    SelectFwe(score_func=f_classif, alpha=0.012),
    LogisticRegression(C=0.5, dual=False, penalty="l2")
)

exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)

0.6394230769230769

In [168]:
exported_pipeline = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, max_features=0.5, min_samples_leaf=2, min_samples_split=11, n_estimators=100, subsample=0.5)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 20)

exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)

0.31971153846153844

In [169]:
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    LogisticRegression(C=5.0, dual=False, penalty="l2")
)

exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)

0.6370192307692307

In [172]:
'''exported_pipeline = make_pipeline(
    make_union(
        ZeroCount(),
        FunctionTransformer(copy)
    ),
    LogisticRegression(C=0.5, dual=False, penalty="l2")
)
exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)'''

0.6322115384615384

In [34]:
from sklearn.decomposition import PCA

In [ ]:
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        StackingEstimator(estimator=DecisionTreeClassifier(criterion="entropy", max_depth=3, min_samples_leaf=11, min_samples_split=7))
    ),
    PCA(iterated_power=9, svd_solver="randomized"),
    LogisticRegression(C=0.1, dual=False, penalty="l2")
)
exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)

In [ ]:
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        FunctionTransformer(copy)
    ),
    MinMaxScaler(),
    LogisticRegression(C=20.0, dual=False, penalty="l2")
)
exported_pipeline.fit(X_train1, y_train)
exported_pipeline.predict(X_test1)
exported_pipeline.score(X_test1, y_test)

In [181]:
# define TPOTClassifier

tpot_config = {
    'sklearn.naive_bayes.GaussianNB': {
    },
    'sklearn.ensemble.GradientBoostingClassifier': {
    },
    'sklearn.linear_model.’':{
    }
}

tpotmodelf = TPOTClassifier(
   generations=20,
   population_size=100,
   #config_dict='TPOT light',
   cv=5,
   verbosity=2,
   config_dict=tpot_config
)

# performing the search for best fit

tpotmodelf.fit(X_test1, y_test)

# exporting best model
tpotmodelf.export('tpot.py')

Optimization Progress:   0%|          | 0/2100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5143144004589788

Generation 2 - Current best internal CV score: 0.5191910499139415

Generation 3 - Current best internal CV score: 0.5191910499139415

Generation 4 - Current best internal CV score: 0.521514629948365

Generation 5 - Current best internal CV score: 0.521514629948365

Generation 6 - Current best internal CV score: 0.521514629948365

Generation 7 - Current best internal CV score: 0.521514629948365

Generation 8 - Current best internal CV score: 0.52877223178428

Generation 9 - Current best internal CV score: 0.52877223178428

Generation 10 - Current best internal CV score: 0.52877223178428

Generation 11 - Current best internal CV score: 0.52877223178428

Generation 12 - Current best internal CV score: 0.52877223178428

Generation 13 - Current best internal CV score: 0.52877223178428

Generation 14 - Current best internal CV score: 0.52877223178428

Generation 15 - Current best internal CV score: 0.52877223178428

Generati

In [35]:
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        make_union(
            FunctionTransformer(copy),
            StackingEstimator(estimator=GaussianNB())
        )
    ),
    StackingEstimator(estimator=MLPClassifier()),
    MLPClassifier()
)

exported_pipeline.fit(X_test1, y_test)
exported_pipeline.predict(X_train1)
exported_pipeline.score(X_test1,y_test)

NameError: name 'StackingEstimator' is not defined

In [ ]:
estimators = [
    ('rf', DecisionTreeClassifier(criterion="entropy", max_depth=3, min_samples_leaf=11, min_samples_split=7)),
    ('svr', MLPClassifier(random_state=30, max_iter=300,alpha=0.1, learning_rate_init=0.001))]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression())
